In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../utils/")
import Toolbox as tb
from Toolbox import *
import pickle # sirve para guardar cualquier objeto binario, inclusi el pipline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
import os

In [ ]:
# Cargar el archivo CSV
data = pd.read_csv("../data/wines_dataset.csv", sep="|")

"""
División de datos en entrenamiento y prueba.

Se toma el 80% de los datos para entrenamiento y el 20% restante para prueba.

Parámetros:
- test_size (float): Proporción del conjunto de prueba (0.2 = 20% de los datos).
- random_state (int): Semilla para la reproducibilidad de la división.

Salida:
- train (DataFrame): Conjunto de entrenamiento.
- test (DataFrame): Conjunto de prueba.
"""
train, test = train_test_split(data, test_size=0.2, random_state=42)

"""
Guardado de los conjuntos de datos en archivos CSV.

Los archivos se almacenan en la carpeta "./data/" con los nombres:
- wines_train.csv → Contiene el 80% de los datos para entrenamiento.
- wines_test.csv → Contiene el 20% de los datos para prueba.

index=False evita que se guarde el índice en los archivos.
"""
# Crear el directorio si no existe
os.makedirs("./data/", exist_ok=True)

train.to_csv(os.path.join("./data/", 'wines_train.csv'), index=False)
test.to_csv(os.path.join("./data/", 'wines_test.csv'), index=False)

In [3]:
# Cargar el conjunto de datos de entrenamiento desde el archivo CSV
df_train = pd.read_csv("./data/wines_train.csv")

# Mostrar el DataFrame cargado
df_train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,class
0,7.9,0.18,0.40,2.20,0.049,38.0,67.0,0.99600,3.33,0.93,11.3,5,red
1,7.1,0.18,0.74,15.60,0.044,44.0,176.0,0.99960,3.38,0.67,9.0,6,white
2,7.6,0.51,0.24,1.20,0.040,10.0,104.0,0.99200,3.05,0.29,10.8,6,white
3,6.0,0.25,0.28,7.70,0.053,37.0,132.0,0.99489,3.06,0.50,9.4,6,white
4,9.0,0.38,0.41,2.40,0.103,6.0,10.0,0.99604,3.13,0.58,11.9,7,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,6.4,0.24,0.50,11.60,0.047,60.0,211.0,0.99660,3.18,0.57,9.3,5,white
5193,6.6,0.22,0.28,12.05,0.058,25.0,125.0,0.99856,3.45,0.45,9.4,5,white
5194,6.6,0.20,0.38,7.90,0.052,30.0,145.0,0.99470,3.32,0.56,11.0,7,white
5195,7.3,0.41,0.29,1.80,0.032,26.0,74.0,0.98889,2.96,0.35,13.0,8,white


In [4]:
# Cargar el conjunto de datos de prueba desde el archivo CSV
df_test = pd.read_csv("./data/wines_test.csv")

# Mostrar el DataFrame cargado
df_test

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,class
0,7.4,0.320,0.27,1.4,0.049,38.0,173.0,0.99335,3.03,0.52,9.3,5,white
1,6.6,0.340,0.24,3.3,0.034,29.0,99.0,0.99031,3.10,0.40,12.3,7,white
2,6.4,0.320,0.35,4.8,0.030,34.0,101.0,0.99120,3.36,0.60,12.5,8,white
3,6.8,0.230,0.32,1.6,0.026,43.0,147.0,0.99040,3.29,0.54,12.5,6,white
4,6.7,0.340,0.26,1.9,0.038,58.0,138.0,0.98930,3.00,0.47,12.2,7,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,7.6,0.285,0.32,14.6,0.063,32.0,201.0,0.99800,3.00,0.45,9.2,5,white
1296,11.6,0.470,0.44,1.6,0.147,36.0,51.0,0.99836,3.38,0.86,9.9,4,red
1297,10.2,0.340,0.48,2.1,0.052,5.0,9.0,0.99458,3.20,0.69,12.1,7,red
1298,6.2,0.460,0.17,1.6,0.073,7.0,11.0,0.99425,3.61,0.54,11.4,5,red


In [5]:
# Genera un resumen descriptivo del DataFrame utilizando la función tb.describe_df(df).
# 
#  Muestra información sobre:
#    - DATA_TYPE: Tipo de dato de cada columna (numérico, categórico, etc.).
#    - MISSINGS (%): Porcentaje de valores nulos en cada columna.
#    - UNIQUE_VALUES: Cantidad de valores únicos en cada columna.
#    - CARDIN (%): Cardinalidad relativa (valores únicos / total de filas).
# 
# Útil para comprender la estructura de los datos antes del preprocesamiento.

tb.describe_df(df_train)

COL_N,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,class
DATA_TYPE,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,object
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,100,177,87,308,194,132,274,951,106,106,102,7,2
CARDIN (%),0.02,0.03,0.02,0.06,0.04,0.03,0.05,0.18,0.02,0.02,0.02,0.0,0.0


In [6]:
# Definición de variables objetivo para el análisis:

# target_clf = "quality" → Problema de Clasificación
#    - La variable `quality` representa la calidad del vino.
#    - Es una variable categórica con 7 valores únicos (multiclase).
#    - Se utilizarán modelos de clasificación, como RandomForestClassifier o XGBoost.
#    - Es necesario aplicar OneHotEncoder a variables categóricas y StandardScaler a las numéricas.

# target_reg = "alcohol" → Problema de Regresión
#    - La variable `alcohol` indica el porcentaje de alcohol en el vino.
#    - Es una variable numérica continua.
#    - Se utilizarán modelos de regresión, como RandomForestRegressor o LinearRegression.
#    - Requiere escalado de las variables numéricas (StandardScaler) y codificación de las categóricas (OneHotEncoder).

# Ambos problemas requieren preprocesamiento adecuado según el tipo de modelo utilizado.

target_clf = "quality"
y_train_cat = df_train["quality"]
y_test_cat = df_test["quality"]
target_reg = "alcohol"
y_train_reg = df_train["alcohol"]
y_test_reg = df_test["alcohol"]


# OPCIÓN REGRESIÓN


In [7]:
# Definición de características (features) para el problema de regresión:

# features_cat_reg → Variables categóricas
#    - Contiene "class" (tipo de vino: tinto o blanco) y "quality" (calidad del vino).
#    - Estas variables deben ser transformadas con OneHotEncoder.

# features_num_reg → Variables numéricas
#    - Se seleccionan todas las columnas del DataFrame excepto las categóricas.
#    - Esto se logra con una lista por comprensión, excluyendo las columnas en features_cat_reg.
#    - Estas variables deben ser escaladas con StandardScaler.

features_cat_reg = ["class", "quality"]
features_num_reg = [col for col in df_train.columns if col not in features_cat_reg]

# Ahora features_num_reg contiene solo las variables numéricas.

In [8]:
# Estudio de correlaciones entre las variables numéricas y la variable objetivo de regresión.

# Calculamos la matriz de correlación solo para las variables numéricas.
#    - Se utiliza df[features_num_reg] para excluir las variables categóricas.
#    - numeric_only="True" se usa para evitar advertencias en versiones recientes de pandas.

# Extraemos la correlación absoluta de cada variable con el target de regresión ("alcohol").
#    - np.abs() se usa para obtener la magnitud de la correlación sin importar el signo.
#    - .sort_values(ascending=False) ordena las variables de mayor a menor correlación.

corr = df_train[features_num_reg].corr(numeric_only="True")
serie_corr = np.abs(corr[target_reg]).sort_values(ascending=False)
serie_corr

# Ahora, serie_corr contiene las variables ordenadas según su grado de correlación con "alcohol".

alcohol                 1.000000
density                 0.682345
residual sugar          0.357459
total sulfur dioxide    0.272970
chlorides               0.260508
free sulfur dioxide     0.188460
pH                      0.116497
fixed acidity           0.091964
volatile acidity        0.036041
citric acid             0.005690
sulphates               0.000412
Name: alcohol, dtype: float64

In [9]:
# Selección de características numéricas basadas en la correlación con el target de regresión.

# Definimos un umbral mínimo de correlación.
#    - r_min = 0.05 significa que solo seleccionaremos variables con correlación mayor a 0.05.

r_min = 0.05

# Seleccionamos las variables numéricas con correlación significativa con el target.
#    - Tomamos solo las variables con correlación absoluta mayor que r_min.
#    - Convertimos los nombres de las columnas en una lista.
#    - Eliminamos el target ("alcohol") de la lista, ya que no es una feature.

features_num_reg_1 = serie_corr[serie_corr > r_min].index.to_list()
features_num_reg_1.remove(target_reg)

# Identificamos las variables numéricas con correlación baja o nula con el target.
#    - Excluimos las variables en features_num_reg_1.
#    - Excluimos el target y las variables categóricas.

features_num_reg_2 = [col for col in df_train.columns if col not in features_num_reg_1 and col != target_reg
                       and col not in features_cat_reg]

# Ahora:
# - features_num_reg_1 contiene las variables más correlacionadas con "alcohol".
# - features_num_reg_2 contiene las variables menos correlacionadas.

In [10]:
features_num_reg_1

['density',
 'residual sugar',
 'total sulfur dioxide',
 'chlorides',
 'free sulfur dioxide',
 'pH',
 'fixed acidity']

In [11]:
# Definición del problema de regresión.

# target_reg → Variable objetivo para regresión.
#    - Se ha definido previamente como "alcohol".
#    - Representa el porcentaje de alcohol en el vino (variable continua).
#    - Es un problema de regresión, ya que el target es numérico y continuo.

target_reg

'alcohol'

In [12]:
# Definición de las características categóricas para el problema de regresión.

# features_cat_reg → Variables categóricas a incluir en el modelo de regresión.
#    - Contiene:
#      - "class" (tipo de vino: tinto o blanco).
#      - "quality" (calidad del vino en una escala categórica).
#    - Ambas variables deben ser transformadas con OneHotEncoder para su uso en modelos numéricos.

features_cat_reg

['class', 'quality']

In [13]:
# Definición de características numéricas principales para regresión.

# features_num_reg_1 → Variables numéricas con mayor correlación con el target de regresión ("alcohol").
#    - Se han seleccionado aquellas con correlación absoluta > r_min (0.05).
#    - Estas variables son las más relevantes para predecir el contenido de alcohol en el vino.
#    - Se recomienda aplicar StandardScaler para normalizar estas características antes del modelado.

features_num_reg_1

['density',
 'residual sugar',
 'total sulfur dioxide',
 'chlorides',
 'free sulfur dioxide',
 'pH',
 'fixed acidity']

In [14]:
# Definición de columnas a incluir y excluir en el modelo de regresión.

# columns_to_keep_reg → Columnas que se mantendrán en el modelo de regresión.
#    - Incluye:
#      - features_num_reg_1 (variables numéricas más correlacionadas con el target).
#      - features_cat_reg (variables categóricas a transformar con OneHotEncoder).

# columns_to_exclude_reg → Columnas que se excluirán del modelo de regresión.
#    - Se obtienen eliminando de df.columns las variables incluidas en columns_to_keep_reg.
#    - Estas columnas no serán utilizadas en el modelo.

columns_to_keep_reg = features_num_reg_1 + features_cat_reg

columns_to_exclude_reg = [col for col in df_train.columns if col not in columns_to_keep_reg]

columns_to_exclude_reg

['volatile acidity', 'citric acid', 'sulphates', 'alcohol']

In [15]:
# Definición de Pipelines para preprocesamiento de datos en regresión.

# cat_pipeline → Preprocesamiento de variables categóricas.
#    - "Impute_Mode": Imputa valores faltantes con la moda (valor más frecuente).
#    - "OHEncoder": Aplica OneHotEncoder, ignorando categorías desconocidas.

# logaritmica → Transformación logarítmica de variables numéricas.
#    - Usa FunctionTransformer con np.log1p para estabilizar distribuciones sesgadas.
#    - feature_names_out="one-to-one" mantiene los nombres originales de las características.

# num_pipeline → Preprocesamiento de variables numéricas.
#    - "Impute_Mean": Imputa valores faltantes con la media.
#    - "logaritmo": Aplica la transformación logarítmica definida antes.
#    - "SScaler": Aplica StandardScaler para normalizar las variables numéricas.

# imputer_step_reg → ColumnTransformer para aplicar los Pipelines según el tipo de variable.
#    - "Process_Numeric": Aplica num_pipeline a features_num_reg_1 (variables numéricas seleccionadas para regresión).
#    - "Process_Categorical": Aplica cat_pipeline a features_cat_reg (variables categóricas seleccionadas para regresión).
#    - "Exclude": Elimina las columnas en columns_to_exclude_reg.
#    - remainder="passthrough": Mantiene cualquier otra columna sin modificar.

# pipe_missings_reg → Pipeline final que aplica el ColumnTransformer imputer_step_reg.

cat_pipeline = Pipeline(
    [("Impute_Mode", SimpleImputer(strategy="most_frequent")),  # Imputación con la moda
     ("OHEncoder", OneHotEncoder(handle_unknown='ignore'))  # Manejar categorías desconocidas
    ]
)

logaritmica = FunctionTransformer(np.log1p, feature_names_out="one-to-one") 
# Esto le indica al Pipeline que el número de características no cambia y que puede usar los nombres originales.

num_pipeline = Pipeline(
    [("Impute_Mean", SimpleImputer(strategy = "mean")), # prevision que en el futuro lleguen datos faltantes
     ("logaritmo", logaritmica),
     ("SScaler", StandardScaler()),
    ]
)

imputer_step_reg = ColumnTransformer(
    [("Process_Numeric", num_pipeline,features_num_reg_1), # feature_numericas seleccionadas para clasificación
     ("Process_Categorical", cat_pipeline, features_cat_reg), # feature_categoriacas seleccionadas para regresión
     ("Exclude", "drop", columns_to_exclude_reg)
    ], remainder = "passthrough"
    )

pipe_missings_reg = Pipeline([("first_stage", imputer_step_reg)])

In [ ]:
### EVALUACION DE MODELOS DE REGRESION
# Definimos los modelos a analizar en el problema de regresión.
#   - RandomForestRegressor
#  - XGBRegressor
# - LGBMRegressor

In [16]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [17]:
# Definición de Pipelines para modelos de regresión.

# randomreg_pipeline → Pipeline para RandomForestRegressor.
#    - "Preprocesado": Aplica el Pipeline de preprocesamiento pipe_missings_reg.
#    - "Modelo": Implementa RandomForestRegressor para realizar la regresión.

# XGB_pipeline → Pipeline para XGBRegressor.
#    - "Preprocesado": Aplica el Pipeline de preprocesamiento pipe_missings_reg.
#    - "Modelo": Implementa XGBRegressor, basado en árboles de decisión con boosting.

# LGBMreg_pipeline → Pipeline para LGBMRegressor.
#    - "Preprocesado": Aplica el Pipeline de preprocesamiento pipe_missings_reg.
#    - "Modelo": Implementa LGBMRegressor, optimizado para grandes volúmenes de datos.

# Validación cruzada con 5 folds.
#    - Se evalúan los tres modelos utilizando cross_val_score.
#    - La métrica utilizada es "neg_mean_absolute_percentage_error" (MAPE negativo).
#    - Se imprime el promedio del error absoluto porcentual y los valores individuales por fold.

randomreg_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_reg),  # Aplicación del preprocesamiento definido
     ("Modelo", RandomForestRegressor())  # Modelo de regresión basado en árboles
    ])

XGB_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_reg),  
     ("Modelo", XGBRegressor())  # Modelo de boosting basado en XGBoost
    ])

LGBMreg_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_reg),
     ("Modelo", LGBMRegressor(verbose=-1))  # Modelo optimizado basado en LightGBM
    ])

# Evaluación de modelos con validación cruzada
for name, pipe in zip(["RandomF", "XGB", "LGBM"], [randomreg_pipeline, XGB_pipeline, LGBMreg_pipeline]):
    resultado_reg = cross_val_score(pipe, df_train, y_train_reg, cv=5, scoring="neg_mean_absolute_percentage_error")
    print(f"{name}: {-np.mean(resultado_reg):.4f}")  # Se invierte el signo para interpretar el MAPE positivo
    print(resultado_reg)  # Resultados individuales de cada fold

RandomF: 0.0274
[-0.02855353 -0.02834254 -0.02683549 -0.02656188 -0.0267983 ]
XGB: 0.0271
[-0.02807295 -0.02796904 -0.02587827 -0.02658056 -0.02686444]
LGBM: 0.0287
[-0.02986407 -0.02965162 -0.02812008 -0.02791631 -0.02802684]


In [18]:
# Búsqueda de hiperparámetros para los modelos de regresión mediante GridSearchCV.
# Se optimizan los siguientes modelos:
#   - RandomForestRegressor
#   - XGBRegressor
#   - LGBMRegressor
#
# Se definen los espacios de búsqueda para cada modelo.

# pipe_random_reg → Espacio de búsqueda para RandomForestRegressor.
#    - "Modelo__n_estimators": Número de árboles en el bosque.
#    - "Modelo__max_depth": Profundidad máxima de los árboles.
#    - "Modelo__max_features": Número de características a considerar en cada división.

pipe_random_reg = {
    "Modelo__n_estimators": [100, 200, 1000],  # Diferentes cantidades de árboles en el bosque
    "Modelo__max_depth": [1, 5, 10, 20],  # Variación en la profundidad máxima de los árboles
    "Modelo__max_features": ["log2", "sqrt", None]  # Diferentes métodos de selección de características
}

# pipe_LightGBM → Espacio de búsqueda para LightGBM.
#    - "Modelo__n_estimators": Número de árboles en el modelo.
#    - "Modelo__num_leaves": Número de hojas en cada árbol.
#    - "Modelo__learning_rate": Tasa de aprendizaje del modelo.

pipe_LightGBM = {
    "Modelo__n_estimators": [50, 100, 200],  # Variación en el número de árboles
    "Modelo__num_leaves": [31, 50, 100],  # Número de hojas en cada árbol
    "Modelo__learning_rate": [0.01, 0.1, 0.2]  # Tasa de aprendizaje del modelo
}

# pipe_xgb_param → Espacio de búsqueda para XGBoost.
#    - "Modelo__n_estimators": Número de árboles en el modelo.
#    - "Modelo__max_depth": Profundidad máxima de los árboles.
#    - "Modelo__learning_rate": Tasa de aprendizaje del modelo.

pipe_xgb_param = {
    'Modelo__n_estimators': [10, 100, 200, 400],  # Variación en la cantidad de árboles
    'Modelo__max_depth': [1, 2, 4, 8],  # Profundidad máxima de los árboles
    'Modelo__learning_rate': [0.1, 0.2, 0.5, 1.0]  # Tasa de aprendizaje
}

# Configuración de la validación cruzada con 5 folds.
cv = 5

# GridSearchCV para RandomForestRegressor.
gs_random = GridSearchCV(
    randomreg_pipeline,
    pipe_random_reg,
    cv=cv,
    scoring="neg_mean_absolute_percentage_error",
    verbose=0,
    n_jobs=-1  # Utiliza todos los núcleos disponibles para acelerar la búsqueda
)

# GridSearchCV para LightGBMRegressor.
gs_LightGBM = GridSearchCV(
    LGBMreg_pipeline,
    pipe_LightGBM,
    cv=cv,
    scoring="neg_mean_absolute_percentage_error",
    verbose=0,
    n_jobs=-1
)

# GridSearchCV para XGBRegressor.
gs_xgb = GridSearchCV(
    XGB_pipeline,
    pipe_xgb_param,
    cv=cv,
    scoring="neg_mean_absolute_percentage_error",
    verbose=0,
    n_jobs=-1
)

# Diccionario con los GridSearch configurados para cada modelo.
pipe_grids_reg = {
    "gs_reg_log": gs_random,
    "gs_LightGBM": gs_LightGBM,
    "gs_xgb": gs_xgb
}

In [19]:
# Ejecución de GridSearchCV para cada modelo en el diccionario pipe_grids_reg.
# 
# - Itera sobre cada modelo y su respectiva búsqueda de hiperparámetros.
# - Ajusta el modelo con el conjunto de entrenamiento (df_train, y_train_reg).
# - GridSearchCV seleccionará automáticamente la mejor combinación de hiperparámetros según la métrica definida.
# - La métrica utilizada es "neg_mean_absolute_percentage_error" (MAPE negativo).
# - Los resultados se almacenan dentro de cada objeto GridSearchCV.

for nombre, grid_search in pipe_grids_reg.items():
    print(f"Entrenando GridSearch para {nombre}...")  # Mensaje informativo
    grid_search.fit(df_train, y_train_reg)  # Ajusta el modelo con la búsqueda de hiperparámetros
    print(f"Mejores parámetros para {nombre}: {grid_search.best_params_}")  # Imprime los mejores hiperparámetros encontrados
    print(f"Mejor score (neg-MAPE) para {nombre}: {grid_search.best_score_:.4f}")  # Muestra el mejor puntaje obtenido
    print("-" * 50)  # Separador para mayor claridad

Entrenando GridSearch para gs_reg_log...
Mejores parámetros para gs_reg_log: {'Modelo__max_depth': 20, 'Modelo__max_features': None, 'Modelo__n_estimators': 1000}
Mejor score (neg-MAPE) para gs_reg_log: -0.0273
--------------------------------------------------
Entrenando GridSearch para gs_LightGBM...
Mejores parámetros para gs_LightGBM: {'Modelo__learning_rate': 0.2, 'Modelo__n_estimators': 200, 'Modelo__num_leaves': 100}
Mejor score (neg-MAPE) para gs_LightGBM: -0.0253
--------------------------------------------------
Entrenando GridSearch para gs_xgb...
Mejores parámetros para gs_xgb: {'Modelo__learning_rate': 0.1, 'Modelo__max_depth': 8, 'Modelo__n_estimators': 400}
Mejor score (neg-MAPE) para gs_xgb: -0.0243
--------------------------------------------------


In [20]:
# Evaluación de los mejores modelos tras la búsqueda de hiperparámetros con GridSearchCV.
#
# - Extrae el mejor score obtenido en validación cruzada para cada modelo en pipe_grids_reg.
# - Convierte los resultados en un DataFrame para facilitar su análisis y comparación.
# - Ordena los modelos en función del mejor score (neg-MAPE), de mayor a menor.
# - Permite identificar qué modelo tuvo mejor rendimiento tras la optimización.

# Extraer el mejor score de cada GridSearchCV y almacenarlo en una lista de tuplas.
best_grids_reg = [(i, j.best_score_) for i, j in pipe_grids_reg.items()]

# Convertir la lista en un DataFrame para facilitar su análisis.
best_grids_reg = pd.DataFrame(best_grids_reg, columns=["Grid", "Best score"]) \
                   .sort_values(by="Best score", ascending=False)  # Ordenar de mejor a peor score

# Mostrar los resultados
best_grids_reg

,Grid,Best score
2,gs_xgb,-0.024311
1,gs_LightGBM,-0.025330
0,gs_reg_log,-0.027334


In [21]:
# Selección del mejor modelo tras la búsqueda de hiperparámetros con GridSearchCV.
#
# - Se toma el modelo con el mejor score obtenido en la validación cruzada.
# - Se extrae el nombre del mejor modelo desde best_grids_reg.
# - Se usa ese nombre para acceder al mejor GridSearchCV almacenado en pipe_grids_reg.
# - El modelo seleccionado se almacenará en best_model_reg para futuras predicciones.

# Extraer el mejor modelo basado en la mejor puntuación obtenida.
best_model_reg = pipe_grids_reg[best_grids_reg.iloc[0, 0]]

# Mostrar el mejor modelo seleccionado
best_model_reg

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Preprocesado',
                                        Pipeline(steps=[('first_stage',
                                                         ColumnTransformer(remainder='passthrough',
                                                                           transformers=[('Process_Numeric',
                                                                                          Pipeline(steps=[('Impute_Mean',
                                                                                                           SimpleImputer()),
                                                                                                          ('logaritmo',
                                                                                                           FunctionTransformer(feature_names_out='one-to-one',
                                                                                                                               func=<ufunc 'log1p'>)),
                                                                                                          ('SScaler',
                                                                                                           StandardScaler())]),
                                                                                          ['density',
                                                                                           'residua...
                                                     max_leaves=None,
                                                     min_child_weight=None,
                                                     missing=nan,
                                                     monotone_constraints=None,
                                                     multi_strategy=None,
                                                     n_estimators=None,
                                                     n_jobs=None,
                                                     num_parallel_tree=None,
                                                     random_state=None, ...))]),
             n_jobs=-1,
             param_grid={'Modelo__learning_rate': [0.1, 0.2, 0.5, 1.0],
                         'Modelo__max_depth': [1, 2, 4, 8],
                         'Modelo__n_estimators': [10, 100, 200, 400]},
             scoring='neg_mean_absolute_percentage_error')

In [22]:
# Guardado del mejor modelo de regresión usando Pickle.
#
# - Se asegura de que el directorio "src/models" exista antes de guardar el modelo.
# - Serializa el mejor modelo encontrado (`best_model_reg`) en un archivo .pkl.
# - El modelo guardado podrá ser cargado posteriormente sin necesidad de volver a entrenarlo.

# Asegurar que el directorio "src/models" exista antes de guardar el modelo.
os.makedirs('src/models', exist_ok=True)

# Guardar el mejor modelo en un archivo .pkl dentro de "models".
with open('src/models/modelo_pipeline_reg.pkl', 'wb') as archivo:
    pickle.dump(best_model_reg, archivo)

# OPCIÓN CLASIFICACIÓN


In [23]:
# Definición de las características (features) utilizadas en la clasificación:

# features_cat_clf → Variables categóricas
#    - Contiene solo la variable "class" (tipo de vino: tinto o blanco).
#    - Se debe transformar con OneHotEncoder para convertirla en variables numéricas.

# features_num_clf_1 → Variables numéricas principales
#    - Incluye medidas químicas clave como "volatile acidity", "chlorides", "density", "pH", etc.
#    - Se recomienda aplicar StandardScaler para normalizar estas características.

# features_num_clf_2 → Variables numéricas adicionales
#    - Contiene "fixed acidity" y "residual sugar".
#    - Estas características pueden ser analizadas por separado o combinadas con las anteriores.

features_cat_clf = ["class"]
features_num_clf_1 = ["volatile acidity", "citric acid", "chlorides", "free sulfur dioxide",
                      "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"]
features_num_clf_2 = ["fixed acidity", "residual sugar"]


In [24]:
# Definición del problema de clasificación.

# target_clf → Variable objetivo para clasificación.
#    - Se ha definido previamente como "quality".
#    - Representa la calidad del vino en una escala categórica.
#    - Es un problema de clasificación **multiclase** (7 categorías posibles).

target_clf

'quality'

In [25]:
# Definición de las características categóricas para el problema de clasificación.

# features_cat_clf → Variables categóricas a incluir en el modelo de clasificación.
#    - Contiene la variable "class" (tipo de vino: tinto o blanco).
#    - Se debe transformar con OneHotEncoder para convertirla en variables numéricas.

features_cat_clf

['class']

In [26]:
# Definición de características numéricas principales para clasificación.

# features_num_clf_1 → Variables numéricas seleccionadas para el modelo de clasificación.
#    - Incluye medidas químicas clave del vino:
#      - "volatile acidity", "citric acid", "chlorides", "free sulfur dioxide",
#      - "total sulfur dioxide", "density", "pH", "sulphates", "alcohol".
#    - Se recomienda aplicar StandardScaler para normalizar estas características antes del modelado.

features_num_clf_1

['volatile acidity',
 'citric acid',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [27]:
# Definición de columnas a incluir y excluir en el modelo de clasificación.

# columns_to_keep_clf → Columnas que se mantendrán en el modelo de clasificación.
#    - Incluye:
#      - features_num_clf_1 (variables numéricas relevantes).
#      - features_cat_clf (variables categóricas a transformar con OneHotEncoder).

# columns_to_exclude_clf → Columnas que se excluirán del modelo de clasificación.
#    - Se obtienen eliminando de df.columns las variables incluidas en columns_to_keep_clf.
#    - Estas columnas no serán utilizadas en el modelo.

columns_to_keep_clf =  features_num_clf_1 + features_cat_clf

columns_to_exclude_clf = [col for col in df_train.columns if col not in columns_to_keep_clf] 

columns_to_exclude_clf


['fixed acidity', 'residual sugar', 'quality']

In [ ]:
# Definición de Pipelines para preprocesamiento de datos en clasificación.

# cat_pipeline → Preprocesamiento de variables categóricas.
#    - "Impute_Mode": Imputa valores faltantes con la moda (valor más frecuente).
#    - "OHEncoder": Aplica OneHotEncoder, ignorando categorías desconocidas, en lugar generar un error

# logaritmica → Transformación logarítmica de variables numéricas.
#    - Usa FunctionTransformer con np.log1p para estabilizar distribuciones sesgadas.
#    - feature_names_out="one-to-one" mantiene los nombres originales de las características.

# num_pipeline → Preprocesamiento de variables numéricas.
#    - "Impute_Mean": Imputa valores faltantes con la media.
#    - "logaritmo": Aplica la transformación logarítmica definida antes.
#    - "SScaler": Aplica StandardScaler para normalizar las variables numéricas.

# imputer_step_cat → ColumnTransformer para aplicar los Pipelines según el tipo de variable.
#    - "Process_Numeric": Aplica num_pipeline a features_num_clf_1 (variables numéricas).
#    - "Process_Categorical": Aplica cat_pipeline a features_cat_clf (variables categóricas).
#    - "Exclude": Elimina las columnas en columns_to_exclude_clf.
#    - remainder="passthrough": Mantiene cualquier otra columna sin modificar.

# pipe_missings_cat → Pipeline final que aplica el ColumnTransformer imputer_step_cat.

cat_pipeline = Pipeline(
    [("Impute_Mode", SimpleImputer(strategy="most_frequent")), 
     ("OHEncoder", OneHotEncoder(handle_unknown='ignore'))  
    ]
)

logaritmica = FunctionTransformer(np.log1p, feature_names_out="one-to-one") 

num_pipeline = Pipeline(
    [("Impute_Mean", SimpleImputer(strategy = "mean")), 
     ("logaritmo", logaritmica),
     ("SScaler", StandardScaler()),
    ]
)

imputer_step_cat = ColumnTransformer(
    [("Process_Numeric", num_pipeline,features_num_clf_1), 
     ("Process_Categorical", cat_pipeline, features_cat_clf), 
     ("Exclude", "drop", columns_to_exclude_clf)],
       remainder = "passthrough"
    )

pipe_missings_cat = Pipeline([("first_stage", imputer_step_cat)])

In [ ]:
### EVALUACIÓN DE MODELOS DE CLASIFICACIÓN

In [29]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Definición de Pipelines para modelos de clasificación.
#
# - Se crean pipelines que combinan preprocesamiento y modelos de clasificación.
# - Cada pipeline aplica primero `pipe_missings_cat`, que maneja datos faltantes y 
# codificación de variables categóricas.
# - Luego, se entrena un modelo de clasificación diferente en cada pipeline:
#    - LogisticRegression (Regresión Logística)
#    - RandomForestClassifier (Bosques Aleatorios)
#    - LGBMClassifier (LightGBM)
#
# - Finalmente, se evalúan los modelos con validación cruzada usando "accuracy" como métrica.

# Pipeline con Regresión Logística
logistic_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_cat),  # Paso de preprocesamiento
     ("Modelo", LogisticRegression(max_iter=10000, class_weight="balanced"))  
    ])

# Pipeline con RandomForestClassifier
random_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_cat),
     ("Modelo", RandomForestClassifier(class_weight="balanced"))
    ])


# Pipeline con LightGBMClassifier
LGBM_pipeline = Pipeline(
    [("Preprocesado", pipe_missings_cat),
     ("Modelo", LGBMClassifier(verbose=-1, class_weight="balanced"))
    ])

# Evaluación de los modelos con validación cruzada
for name, pipe in zip(["logistic","randomF","LGBM"], 
                      [logistic_pipeline, random_pipeline, LGBM_pipeline]):
    resultado = cross_val_score(pipe, df_train, y_train_cat, cv=5, scoring="accuracy")
    
    # Mostrar el desempeño de cada modelo
    print(f"{name}: {np.mean(resultado):.4f}")
    print(resultado)

logistic: 0.3323
[0.31826923 0.325      0.35226179 0.32435034 0.34167469]
randomF: 0.6644
[0.67019231 0.67115385 0.66313763 0.65928778 0.65832531]
LGBM: 0.6173
[0.61634615 0.59903846 0.62271415 0.62560154 0.62271415]


In [ ]:
# Evaluación de hiperparámetros para modelos de clasificación con GridSearchCV.
#
# - Se definen diferentes conjuntos de hiperparámetros para los modelos:
#   - Regresión Logística (LogisticRegression)
#   - Random Forest (RandomForestClassifier)
#   - LightGBM (LGBMClassifier)
#
# - Se aplica GridSearchCV para encontrar la mejor combinación de hiperparámetros.
# - Se usa validación cruzada con 5 particiones (cv=5) y "accuracy" como métrica de 
# evaluación.
# - Se almacena cada búsqueda en un diccionario `pipe_grids_cat`.

# Definimos sus hiperparametros
reg_log_param = {"Modelo__penalty": [None,"l2"],
                 "Modelo__C": np.logspace(0, 4, 10),
                 "Modelo__class_weight": [None,"balanced"]}

rand_forest_param = {
    'Modelo__n_estimators': [10, 100, 200, 400],
    'Modelo__max_depth': [None,1,2,4,8],
    'Modelo__max_features': ['sqrt', 1, 2, 3],
    'Modelo__class_weight': [None,'balanced']}

param_grid_lgbm = {
    'Modelo__num_leaves': [15, 31, 50],
    'Modelo__learning_rate': [0.01, 0.05, 0.1],
    'Modelo__n_estimators': [50, 100, 200],
    'Modelo__max_depth': [-1,5, 10, 15],
    'Modelo__class_weight': [None,'balanced']}

cv = 5

gs_reg_log = GridSearchCV(logistic_pipeline,
                            reg_log_param,
                            cv=cv,
                            scoring="accuracy",
                            verbose=1,
                            n_jobs=-1)

gs_rand_forest = GridSearchCV(random_pipeline,
                            rand_forest_param,
                            cv=cv,
                            scoring="accuracy",
                            verbose=1,
                            n_jobs=-1)

gs_lgb = GridSearchCV(LGBM_pipeline,
                        param_grid_lgbm,
                        cv=cv,
                        scoring="accuracy",
                        verbose=1,
                        n_jobs=-1)

pipe_grids_cat = {"gs_reg_log":gs_reg_log,
         "gs_rand_forest":gs_rand_forest,
         "gs_lgb":gs_lgb}

In [32]:
# Ejecución de GridSearchCV para cada modelo de clasificación.
# 
# - Se entrena cada búsqueda de hiperparámetros con los datos de entrenamiento (df_train, y_train_cat).
# - GridSearchCV explorará todas las combinaciones de hiperparámetros definidas previamente.
# - Se utiliza validación cruzada para evaluar el rendimiento de cada combinación.
# - Los mejores modelos serán seleccionados para su uso posterior.

for nombre, grid_search in pipe_grids_cat.items():
    print(f"Entrenando GridSearch para {nombre}...")  # Mensaje informativo
    grid_search.fit(df_train, y_train_cat)  # Ajusta el modelo con la búsqueda de hiperparámetros
    print(f"Finalizado: {nombre}")  # Mensaje de finalización
    print(f"Mejores parámetros para {nombre}: {grid_search.best_params_}")  # Imprime los mejores hiperparámetros encontrados
    print(f"Mejor score para {nombre}: {grid_search.best_score_:.4f}")  # Muestra el mejor puntaje obtenido
    print("-" * 50)  # Separador para mayor claridad

Entrenando GridSearch para gs_reg_log...
Fitting 5 folds for each of 40 candidates, totalling 200 fits
Finalizado: gs_reg_log
Mejores parámetros para gs_reg_log: {'Modelo__C': 59.94842503189409, 'Modelo__class_weight': None, 'Modelo__penalty': 'l2'}
Mejor score para gs_reg_log: 0.5472
--------------------------------------------------
Entrenando GridSearch para gs_rand_forest...
Fitting 5 folds for each of 160 candidates, totalling 800 fits
Finalizado: gs_rand_forest
Mejores parámetros para gs_rand_forest: {'Modelo__class_weight': 'balanced', 'Modelo__max_depth': None, 'Modelo__max_features': 'sqrt', 'Modelo__n_estimators': 100}
Mejor score para gs_rand_forest: 0.6706
--------------------------------------------------
Entrenando GridSearch para gs_lgb...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Finalizado: gs_lgb
Mejores parámetros para gs_lgb: {'Modelo__class_weight': None, 'Modelo__learning_rate': 0.1, 'Modelo__max_depth': 15, 'Modelo__n_estimators': 200, 'Mode

In [33]:
# Evaluación de los mejores modelos de clasificación
#
# - Se extrae el mejor puntaje (best_score_) de cada modelo tras la búsqueda de hiperparámetros.
# - Se almacena en un DataFrame para facilitar la comparación.
# - Se ordenan los modelos en función de su desempeño, de mayor a menor.

best_grids_cat = [(i, j.best_score_) for i, j in pipe_grids_cat.items()]  # Extrae el mejor puntaje de cada modelo

best_grids_cat = pd.DataFrame(best_grids_cat, columns=["Grid", "Best score"]).sort_values(by="Best score", ascending=False)  # Crea un DataFrame ordenado

best_grids_cat  # Muestra los resultados

,Grid,Best score
1,gs_rand_forest,0.670577
2,gs_lgb,0.653261
0,gs_reg_log,0.547240


In [ ]:
# Selección del mejor modelo de clasificación
#
# - Se extrae el modelo con mejor rendimiento según la evaluación de GridSearchCV.
# - Se utiliza la primera fila del DataFrame `best_grids_cat`, que ya está ordenado por desempeño.
# - Este modelo se almacenará en la variable `best_model_cat` para su posterior uso o guardado.
best_model_cat = pipe_grids_cat[best_grids_cat.iloc[0, 0]]  # Extrae el mejor modelo
best_model_cat  # Muestra el modelo seleccionado

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Preprocesado',
                                        Pipeline(steps=[('first_stage',
                                                         ColumnTransformer(remainder='passthrough',
                                                                           transformers=[('Process_Numeric',
                                                                                          Pipeline(steps=[('Impute_Mean',
                                                                                                           SimpleImputer()),
                                                                                                          ('logaritmo',
                                                                                                           FunctionTransformer(feature_names_out='one-to-one',
                                                                                                                               func=<ufunc 'log1p'>)),
                                                                                                          ('SScaler',
                                                                                                           StandardScaler())]),
                                                                                          ['volatile '
                                                                                           'acidity...
                                                                                          ['class']),
                                                                                         ('Exclude',
                                                                                          'drop',
                                                                                          ['fixed '
                                                                                           'acidity',
                                                                                           'residual '
                                                                                           'sugar',
                                                                                           'quality'])]))])),
                                       ('Modelo',
                                        RandomForestClassifier(class_weight='balanced'))]),
             n_jobs=-1,
             param_grid={'Modelo__class_weight': [None, 'balanced'],
                         'Modelo__max_depth': [None, 1, 2, 4, 8],
                         'Modelo__max_features': ['sqrt', 1, 2, 3],
                         'Modelo__n_estimators': [10, 100, 200, 400]},
             scoring='accuracy', verbose=1)

In [35]:
# Guardado del mejor modelo de clasificación en formato pickle
#
# - Se asegura de que el directorio `src/models` exista para almacenar el modelo.
# - Se guarda el modelo en un archivo .pkl para su uso posterior.
# - El modelo guardado podrá ser cargado y utilizado sin necesidad de volver a entrenarlo.

# Asegurar que el directorio de modelos exista
os.makedirs('src/models', exist_ok=True)

# Guardar el mejor modelo en formato pickle
with open('src/models/modelo_pipeline_cat.pkl', 'wb') as archivo:
    pickle.dump(best_model_cat, archivo)  # Guarda el modelo en el archivo